<a href="https://colab.research.google.com/github/uzairname/OtsegoStoryProject/blob/main/experiments/Final_Analysis_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you would like to view the data, it is available at https://docs.google.com/spreadsheets/d/1WMoeCGweQA0xUtEwlK8R_XzRuZOgPqpHQDGWizb9slQ/

To be able to use Gemma, follow the instructions that show up after running the cell below to create a hugging face API key, and enter it when prompted.

If asked to add the token as a git credential, type n.

In [ ]:
# !huggingface-cli login

In [ ]:
!pip install -U bertopic bitsandbytes accelerate -q

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from datetime import timedelta
from sklearn.linear_model import LinearRegression
import numpy as np
import random
import re
from nltk.tokenize import sent_tokenize
from tqdm.notebook import tqdm

np.random.seed(42)
random.seed(42)

tqdm.pandas()

# -------------------------------
# 1. Data Loading & Preprocessing
# -------------------------------
nltk.download('vader_lexicon')

# Load CSV
csv_url = f'https://docs.google.com/spreadsheets/d/1WMoeCGweQA0xUtEwlK8R_XzRuZOgPqpHQDGWizb9slQ/export?format=csv'
df = pd.read_csv(csv_url)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df[df['timestamp'].notnull()]  # Remove rows with invalid timestamps
df['content'] = df['content'].fillna("").astype(str)

nltk.download('punkt_tab')

def preprocess_text(text):

    # Replace newlines with a period and a space
    text = re.sub(r'[\r\n]+', '. ', text)

    # Tokenize into sentences
    sentences = sent_tokenize(text)

    # Filter out short sentences
    filtered_sentences = [s for s in sentences if len(s) >= 20]

    # Remove unwanted characters but preserve punctuation: , " . ? !
    cleaned_sentences = [
        re.sub(r'[^a-zA-Z0-9\s,.!?\"\'’]', '', s)
        for s in filtered_sentences
    ]

    # Optionally: lowercase everything for consistency
    cleaned_sentences = [s.lower() for s in cleaned_sentences]

    return ' '.join(cleaned_sentences)

# Remove "See more"
df['content'] = df['content'].str.replace("\nSee more", "", regex=False)

# Apply the preprocessing function to the content column
df['cleaned_content'] = df['content'].apply(preprocess_text)

# remove empty rows
df = df[df['cleaned_content'] != '']

# Inspect the first few rows of the new column
print(df[['content', 'cleaned_content']].head())

In [ ]:
df[(df['content'].str.len() < 21)]

### Apply Sentiment Analysis

The following cell computes sentiment scores via BERT and Vader. It may take a few minutes

In [ ]:
# -------------------------------
# 2. Sentiment Analysis: VADER
# -------------------------------
vader = SentimentIntensityAnalyzer()
df['vader_compound'] = df['content'].apply(lambda x: vader.polarity_scores(x)['compound'])

# -------------------------------
# 3. Sentiment Analysis: BERT
# -------------------------------
# Initialize a BERT sentiment-analysis pipeline.


bert_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

def get_bert_compound(text):
    # Get the sentiment result (returns a list of dicts)
    result = bert_pipeline(text)
    # Extract the star rating from the label (e.g., "4 stars")
    label = result[0]['label']
    rating = int(label.split()[0])
    # Map rating (1-5) to a compound score between -1 and 1:
    # 1 -> -1.0, 2 -> -0.5, 3 -> 0.0, 4 -> 0.5, 5 -> 1.0
    compound = (rating - 3) / 2.0
    return compound

# Apply BERT sentiment analysis (this may take a bit, depending on the dataset size)
# df['bert_compound'] = df['content'].progress_apply(get_bert_compound)

sentiment_pipeline = pipeline("sentiment-analysis")

df['sentiment'] = df['content'].progress_apply(lambda x: sentiment_pipeline(x)[0]['label'])

# Fit Topic Model

The following cells train a topic model on the posts. It assigns a topic index to each post. We can then analyze the posts by topic, date, and sentiment.

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import HDBSCAN
from bertopic.representation import KeyBERTInspired
from umap import UMAP

model = BERTopic(
    embedding_model="all-MiniLM-L6-v2",
    top_n_words=10,
    min_topic_size=10,
    n_gram_range=(1, 2),
    vectorizer_model=CountVectorizer(ngram_range=(1, 2), stop_words="english"),
    representation_model=KeyBERTInspired(),
    hdbscan_model= HDBSCAN(min_cluster_size=20),
    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine'),
    verbose=True,
    nr_topics=6,
)

topics, probs = model.fit_transform(df['cleaned_content'])
df['topic'] = topics


# Generate Topic Names

The following cells will use a language model to assign the topics short titles in natural language. This is a free, open source model from hugging face. It may take a minute to download the model.

In [ ]:
from transformers import pipeline

# client = OpenAI()
# def chat(prompt):
#   response = client.responses.create(
#       model="gpt-4o-mini",
#       input=prompt
#   )
#.  return response.output_text

pipe = pipeline("text-generation", model="google/gemma-3-1b-it")

In [ ]:
def chat(prompt):
  from transformers import pipeline

  messages = [
      {"role": "user", "content": prompt},
  ]
  output = pipe(messages)
  return output[0]['generated_text'][-1]['content']

def get_topic_info(model, topic_id, df, n_docs=10):
  topic_docs = df[df['topic']==topic_id]
  topic_docs = topic_docs.sample(min(n_docs, len(topic_docs)))
  keywords = ', '.join([i for i in model.generate_topic_labels(nr_words=5) if int(i.split('_')[0]) == topic_id][0].split('_')[1:])
  return topic_docs['content'].tolist(), keywords


In [ ]:

topic_names = {}
for topic_id in list(set(model.topics_)):
  docs, keywords = get_topic_info(model, topic_id, df)

  docs_str = "\n".join(docs)

  prompt = docs_str + f"\n\nYou are an AI topic model. Above are a subset of facebook posts from a community environmental health advocacy group in Otsego, MI that follow a common theme. The theme has the key words \"{keywords}\". Please come up with a clear and accurate name representing the subject of what people are discussing in 5 words or less. Respond with just the name and nothing else."

  print(prompt)

  response = chat(prompt)
  print(response)

  topic_names[topic_id]=response.strip()

df['topic_name'] = df['topic'].map(topic_names)


In [ ]:
df['topic_name'].value_counts()

# Figures

## Sentiment by topic

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Determine mean sentiment by topics
df.groupby('topic_name').agg({'vader_compound': 'mean'})

# Determine mean sentiment by topics over time
sentiment_by_topic_year = df.groupby(['topic_name', df['timestamp'].dt.year]).agg({'vader_compound': 'mean', 'bert_compound': 'mean'})

sentiment_by_topic_year = sentiment_by_topic_year.reset_index()

# Create the Plotly line plot
fig = px.line(
    sentiment_by_topic_year,
    x='timestamp',
    y=['vader_compound'],
    color='topic_name',
    title='Mean Sentiment by Topic Over Time',
    labels={'timestamp': 'Year', 'value': 'Mean Sentiment', 'topic_name': 'Topic'},
)

fig.show()

In [ ]:
# -------------------------------
# 4. Daily Aggregation & Forecasting
# -------------------------------
# Create a date column
df['date'] = df['timestamp'].dt.date

# Aggregate daily average sentiment for VADER and BERT
daily_vader = df.groupby('date')['vader_compound'].mean().reset_index().rename(columns={'vader_compound': 'avg_compound'})
daily_bert = df.groupby('date')['bert_compound'].mean().reset_index().rename(columns={'bert_compound': 'avg_compound'})
df['day_mean_vader'] = df.groupby('date')['vader_compound'].transform('mean')
df['day_mean_bert'] = df.groupby('date')['bert_compound'].transform('mean')


# Forecast function using linear regression
def forecast_sentiment():
    # Convert dates to ordinal numbers for regression
    daily_df['date_ordinal'] = pd.to_datetime(daily_df['date']).apply(lambda date: date.toordinal())
    X = daily_df['date_ordinal'].values.reshape(-1, 1)
    y = daily_df['avg_compound'].values
    model = LinearRegression()
    model.fit(X, y)

    # Forecast from the day after the last date until the end of 2026
    last_date = pd.to_datetime(daily_df['date'].max())
    future_dates = pd.date_range(start=last_date + timedelta(days=1), end=pd.Timestamp("2026-12-31"))
    future_ordinals = np.array([d.toordinal() for d in future_dates]).reshape(-1, 1)
    predicted = model.predict(future_ordinals)
    future_df = pd.DataFrame({'date': future_dates, 'predicted_compound': predicted})
    return future_df

vader_forecast = forecast_sentiment(df[['date', 'day_mean_vader']].rename(columns={'day_mean_vader': 'avg_compound'}))
bert_forecast = forecast_sentiment(df[['date', 'day_mean_bert']].rename(columns={'day_mean_bert': 'avg_compound'}))

# -------------------------------
# 5. Visualization
# -------------------------------

# Figure 1: Trend Plots (side-by-side) for VADER and BERT with Forecasts
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# VADER Plot
axs[0].plot(daily_vader['date'], df['avg_compound'], marker='o', label='Historical')
axs[0].plot(vader_forecast['date'], vader_forecast['predicted_compound'], marker='x', linestyle='--', label='Forecast')
axs[0].set_xlabel('Date', fontsize=12)
axs[0].set_ylabel('Avg Compound Sentiment', fontsize=12)
axs[0].set_title('VADER Sentiment Trend & Forecast', fontsize=14)
axs[0].set_ylim(-1, 1)
axs[0].axhline(y=0.05, color='gray', linestyle='--', linewidth=1)
axs[0].axhline(y=-0.05, color='gray', linestyle='--', linewidth=1)
axs[0].grid(True)
axs[0].legend(fontsize=10)
axs[0].text(daily_vader['date'].iloc[-1], 0.07, 'Positive', color='green', fontsize=10)
axs[0].text(daily_vader['date'].iloc[-1], 0.00, 'Neutral', color='blue', fontsize=10)
axs[0].text(daily_vader['date'].iloc[-1], -0.09, 'Negative', color='red', fontsize=10)

# BERT Plot
axs[1].plot(daily_bert['date'], daily_bert['avg_compound'], marker='o', label='Historical')
axs[1].plot(bert_forecast['date'], bert_forecast['predicted_compound'], marker='x', linestyle='--', label='Forecast')
axs[1].set_xlabel('Date', fontsize=12)
axs[1].set_title('BERT Sentiment Trend & Forecast', fontsize=14)
axs[1].set_ylim(-1, 1)
axs[1].axhline(y=0.05, color='gray', linestyle='--', linewidth=1)
axs[1].axhline(y=-0.05, color='gray', linestyle='--', linewidth=1)
axs[1].grid(True)
axs[1].legend(fontsize=10)
axs[1].text(daily_bert['date'].iloc[-1], 0.07, 'Positive', color='green', fontsize=10)
axs[1].text(daily_bert['date'].iloc[-1], 0.00, 'Neutral', color='blue', fontsize=10)
axs[1].text(daily_bert['date'].iloc[-1], -0.09, 'Negative', color='red', fontsize=10)

plt.tight_layout()
plt.show()

# Figure 2: Scatter Plot Comparing VADER vs. BERT for Each Post
plt.figure(figsize=(8, 6))
plt.scatter(df['vader_compound'], df['bert_compound'], alpha=0.6)
plt.xlabel('VADER Compound Score', fontsize=12)
plt.ylabel('BERT Compound Score', fontsize=12)
plt.title('VADER vs. BERT Sentiment Scores', fontsize=14)
plt.grid(True)
lims = [-1, 1]
plt.plot(lims, lims, 'r--', linewidth=1)  # Diagonal reference line
plt.xlim(lims)
plt.ylim(lims)
plt.tight_layout()
plt.show()

# Figure 3: Histograms of Sentiment Distributions for VADER and BERT
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

ax1.hist(df['vader_compound'], bins=20, color='skyblue', edgecolor='black')
ax1.set_title('VADER Sentiment Distribution', fontsize=14)
ax1.set_xlabel('VADER Compound Score', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_xlim(-1, 1)
ax1.grid(True)

ax2.hist(df['bert_compound'], bins=20, color='salmon', edgecolor='black')
ax2.set_title('BERT Sentiment Distribution', fontsize=14)
ax2.set_xlabel('BERT Compound Score', fontsize=12)
ax2.set_xlim(-1, 1)
ax2.grid(True)

plt.tight_layout()
plt.show()

# -------------------------------
# 6. Summary Statistics & Analysis Output
# -------------------------------

In [ ]:
# Additional Visualizations

import matplotlib.pyplot as plt

# --- Visualization 4: Box Plots of VADER and BERT Compound Scores by Day of Week ---

# Create a 'day_of_week' column in the DataFrame with proper ordering
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df['day_of_week'] = pd.Categorical(df['timestamp'].dt.day_name(), categories=days_order, ordered=True)

# Prepare the data lists for each day for both VADER and BERT scores
vader_box_data = [df.loc[df['day_of_week'] == day, 'vader_compound'].dropna() for day in days_order]
bert_box_data = [df.loc[df['day_of_week'] == day, 'bert_compound'].dropna() for day in days_order]

# Create side-by-side box plots
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

axs[0].boxplot(vader_box_data, labels=days_order)
axs[0].set_title("VADER Compound Scores by Day of Week")
axs[0].set_xlabel("Day of Week")
axs[0].set_ylabel("Compound Score")
axs[0].grid(True)

axs[1].boxplot(bert_box_data, labels=days_order)
axs[1].set_title("BERT Compound Scores by Day of Week")
axs[1].set_xlabel("Day of Week")
axs[1].grid(True)

plt.tight_layout()
plt.show()


# --- Visualization 5: Stacked Bar Chart of VADER Sentiment Categories by Day of Week ---

# Define a helper function to assign sentiment categories for VADER
def categorize(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the function to create a new category column for VADER scores
df['vader_category'] = df['vader_compound'].apply(categorize)

# Create a pivot table: counts of sentiment categories by day of week
pivot = df.groupby('day_of_week')['vader_category'].value_counts().unstack().fillna(0).loc[days_order]

# Plot a stacked bar chart for the VADER sentiment categories by day
pivot.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
plt.title("VADER Sentiment Categories by Day of Week")
plt.xlabel("Day of Week")
plt.ylabel("Number of Posts")
plt.legend(title="Sentiment Category")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

print("Summary Statistics:")

total_posts = len(df)
print(f"Total posts analyzed: {total_posts}")

avg_vader = df['vader_compound'].mean()
avg_bert = df['bert_compound'].mean()
std_vader = df['vader_compound'].std()
std_bert = df['bert_compound'].std()

print(f"Average VADER compound score: {avg_vader:.3f}")
print(f"Average BERT compound score: {avg_bert:.3f}")
print(f"Standard Deviation VADER compound score: {std_vader:.3f}")
print(f"Standard Deviation BERT compound score: {std_bert:.3f}")

# Define a helper function to assign a sentiment category
def categorize(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

vader_categories = df['vader_compound'].apply(categorize)
bert_categories = df['bert_compound'].apply(categorize)

vader_counts = vader_categories.value_counts()
bert_counts = bert_categories.value_counts()

print("\nVADER Sentiment Category Counts:")
print(vader_counts)
print("\nBERT Sentiment Category Counts:")
print(bert_counts)

corr = df['vader_compound'].corr(df['bert_compound'])
print(f"\nCorrelation between VADER and BERT compound scores: {corr:.3f}")

# Sentiment Analysis Summary

**Total Posts Analyzed:** 256

## Overall Sentiment
- **VADER Average Compound:** 0.086  
- **BERT Average Compound:** 0.092  
*Both methods show slightly positive averages, but these can mask underlying complexity.*

- **VADER Std. Dev.:** 0.449  
- **BERT Std. Dev.:** 0.800  
*BERT exhibits a wider range, indicating more extreme sentiment values.*

## Sentiment Categories
- **VADER Counts:**
  - Positive: 110
  - Neutral: 84
  - Negative: 62

- **BERT Counts:**
  - Positive: 117
  - Negative: 91
  - Neutral: 48

*BERT flags more negative posts, suggesting higher sensitivity to distress.*

## Correlation Between Methods
- **Correlation:** 0.437  
*Moderate correlation indicates the two methods capture sentiment differently.*

## Insights & Recommendations
- **Context Matters:**  
  The slightly positive averages hide variability. BERT’s higher negative count may better reflect community distress.
  
- **Multiple Methods:**  
  Using both VADER and BERT provides a fuller picture. Consider fine-tuning BERT on domain-specific data for improved accuracy.

- **Additional Visualizations:**  
  - **Heatmaps/Calendar Plots:** Identify periods with heightened negative sentiment.  
  - **Topic Modeling:** Use LDA to link themes (e.g., water quality, cancer) with sentiment trends.  
  - **Event Annotations:** Overlay key community events on trend graphs to contextualize shifts in sentiment.

## Conclusion
While overall sentiment appears slightly positive, the variability and methodological differences (especially BERT’s detection of more negative sentiment) suggest that the community's emotional response is more nuanced and polarized. This underscores the need for a combined quantitative and qualitative approach to fully capture the community's experiences, particularly given the serious issues of water quality and health.

